In [1]:
import os
import numpy as np
import pandas as pd
import functions as func

from sklearn.model_selection import train_test_split
from functools import partial
from concurrent.futures import ProcessPoolExecutor, Executor, as_completed

In [2]:
def read_data(path, filename):
    print(f'reading file = {os.path.join(path, filename)}')
    data = pd.read_csv(os.path.join(path, filename))
    data = data.rename(columns={'Unnamed: 0':'Id'})
    print(f'data shape = {data.shape}')
    types_info = pd.DataFrame(data.dtypes.value_counts(), columns=['columns_count'])
    print('types info about df columns: ')
    print(types_info)
    return data

In [3]:
ls -la -h Datasets/GiveMeSomeCredit/

total 28272
drwx------@ 6 adam  staff   192B Dec 23 21:30 ./
drwxr-xr-x  8 adam  staff   256B Dec 23 21:36 ../
-rwxr-xr-x@ 1 adam  staff    15K Dec 11  2019 Data Dictionary.xls*
-rwxr-xr-x@ 1 adam  staff   4.8M Dec 11  2019 cs-test.csv*
-rwxr-xr-x@ 1 adam  staff   7.2M Dec 11  2019 cs-training.csv*
-rwxr-xr-x@ 1 adam  staff   1.8M Dec 11  2019 sampleEntry.csv*


In [4]:
train_data = read_data(path='Datasets/GiveMeSomeCredit/', filename='cs-training.csv')
# test_data = read_data(path='Datasets/GiveMeSomeCredit/', filename='cs-test.csv')
# descript = pd.read_excel("Datasets/GiveMeSomeCredit/Data Dictionary.xls")
# sample_data = pd.read_csv("Datasets/GiveMeSomeCredit/sampleEntry.csv")

reading file = Datasets/GiveMeSomeCredit/cs-training.csv
data shape = (150000, 12)
types info about df columns: 
         columns_count
int64                8
float64              4


In [5]:
def transform_to_description(data: pd.DataFrame):
    transformed_data = pd.DataFrame(columns=data.columns)
    
    for col in data:
        transformed_data[col] = data[col].apply(lambda x: (x, x))
    
    return transformed_data

In [6]:
train_data.fillna(0, inplace=True)
# test_data.fillna(0, inplace=True)

#ставим колонку Id как индекс клиента
train_data.set_index('Id', inplace=True)
# test_data.set_index('Id', inplace=True)

#сохраняем метку класса
train_label = train_data['SeriousDlqin2yrs'].copy()
train_data.drop('SeriousDlqin2yrs', axis=1, inplace=True)
#удаляем колонку класса из тестовых данных, так как она не несет никакой информации
# test_data.drop('SeriousDlqin2yrs', axis=1, inplace=True)

In [7]:
train_data['NumberOfDependents'] = train_data.NumberOfDependents.astype('int')
train_data['MonthlyIncome'] = train_data.MonthlyIncome.astype('int')

In [26]:
train_data.shape#, test_data.shape

(150000, 10)

In [8]:
float_cols = train_data.select_dtypes('float').columns
train_data.loc[:, float_cols] = train_data.loc[:, float_cols].round(2)
# train_data.head()

In [28]:
# float_cols = test_data.select_dtypes('float').columns
# test_data.loc[:, float_cols] = test_data.loc[:, float_cols].round(2)
# test_data.head()

In [9]:
transformed_train = transform_to_description(train_data)
# transformed_test = transform_to_description(test_data)
# transformed_train.head(5)

In [10]:
transformed_train.shape#, transformed_test.shape

(150000, 10)

In [11]:
trainX, testX, trainY, testY = train_test_split(transformed_train, train_label, test_size=0.4)

In [12]:
valX, testX, valY, testY = train_test_split(testX, testY, test_size=0.5)

In [13]:
print(trainX.shape, valX.shape, testX.shape)
print(trainY.shape, valY.shape, testY.shape)

(90000, 10) (30000, 10) (30000, 10)
(90000,) (30000,) (30000,)


In [14]:
def similarity(vect1: pd.Series, vect2:pd.Series):
    
    """
    previous version:
     vect1 = transformed_train.iloc[0]
     vect2 = transformed_train.iloc[2]
     func = (lambda x,y: (min(x[0], y[0]), max(x[1], y[1])))
     pd.Series(map(func, vect1, vect2), index=train_data.columns)
    
     for col in cols:
     vect_min = min(vect1.loc[col][0], vect2.loc[col][0])
     vect_max = max(vect1.loc[col][1], vect2.loc[col][1])
     vect[col] = (vect_min, vect_max)
    """
    
    func = lambda x, y: (min(x[0], y[0]), max(x[1], y[1]))
    vect = pd.Series(map(func, vect1, vect2), index=vect1.index)
    return vect

def inclusion(obj, patterns):
    """
    check where an obj is inluded in list of patterns
    is_include = any([all(obj == elem) for elem in patterns])
    """     
    is_include = any([obj.equals(elem) for elem in patterns])
    return  is_include

**Алгоритм из работы Алексея(QBCA)**

In [16]:
alpha = 0.005
sample_ratio = 0.01
num_iters = 100
# N_neg = train_label.value_counts().reset_index().iloc[0, 1]
# N_pos = train_label.value_counts().reset_index().iloc[1, 1]
# N_neg, N_pos

**не обновляем индекс так как индекс - это id клиента, имеет значимую информацию**

In [17]:
# train_pos = transformed_train.loc[train_label[train_label == 1].index]
# train_neg = transformed_train.loc[train_label[train_label == 0].index]
train_pos = trainX.loc[trainY[trainY == 1].index]
train_neg = trainX.loc[trainY[trainY == 0].index]

train_neg.shape, train_pos.shape

((84054, 10), (5946, 10))

**Берем выборку заемов, для начального тестирования работоспособности**

In [18]:
train_pos = train_pos.sample(n=1000, random_state=123, replace=False)
train_neg = train_neg.sample(n=1000, random_state=123, replace=False)
train_pos.shape, train_neg.shape

((1000, 10), (1000, 10))

In [19]:
test_sample = valX.sample(1)
test_sample

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
Id,,,,,,,,,,
141155,"(0.01, 0.01)","(34, 34)","(0, 0)","(0.11, 0.11)","(3750, 3750)","(4, 4)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"


**get_similarity_sample_repr and is_included_in_repr работают вроде правильно**

По крайней мере на тестовых объектах показали правильные результаты

In [20]:
def get_similarity_sample_repr(sample: pd.DataFrame):
    
    """
    get sample of feature represantations from pos or neg class dataset
    returns feature represantation for sample by similarity operation
    """
    pattern = None
    for i, obj in sample.iterrows():
        if pattern is None:
            pattern = obj
        else:
            pattern = similarity(pattern, obj)
    return pattern
    
#операция нахождения объектов по признаковому представлению
def is_included_in_repr(d: pd.Series, train_data: pd.DataFrame):
    """
    returns objects from train dataset(from train pos and neg data) that is included in d representation
    """
    
    d_list = []
    
    for i, obj in train_data.iterrows():
        feature_repr = similarity(obj, d)
        is_included = d.equals(feature_repr)
        if is_included:
            d_list.append(obj)
    d_list = pd.DataFrame(d_list) if len(d_list) > 0 else None
    return d_list

### Mining step


    для положительного класса нас интересуют объекты отрицательного класса, 
    а для отрицательного - положительные
    Уже после определения объектов из другого класса, попадающие в признаковое представление семпла данных, будет приниматься решение
     о включение этого признакого представление в список гипотез представления(областей или интервальных представлений)
    

Что нужно посмотреть:

    1. Сравнение количества генерируемых гипотез в зависимости от критерия и области генерации выборки.
    (Возможно изобразить всего 4 варианта: старый подход - локальная или случайная выборка, новый подход - 
    локальная или случайная выборка)
    
    2. Сравнение генерируемых гипотез для старого и нового критерия в зависимости от ширины локальной области генерации
    (Возможно есть какая то оптимальная ширина окна)
    

To-do-list:
1. **классификации при разном включении признаков из исходного множества**

2. **Подумать о том, каким образом генерить гипотезы. То есть выбирать не случайно выборку из множества объектов,
    а какую-то локальную область. (Делать будем через расширение области объекта; более того, будем искать 
    оптимальное значение расширения локальной области)**

In [ ]:
def generate_random_sample(train_data: pd.DataFrame, sample_size: int, d: pd.Series = None):
    
    if d is not None:
        local_objects = is_included_in_repr(d=d, train_data=train_data)
        inds = np.random.RandomState().choice(local_objects.index, replace=False, size=sample_size)
        sample = local_objects.loc[inds]
    else:
        inds = np.random.RandomState().choice(train_data.index, replace=False, size=sample_size)
        sample = train_data.loc[inds].copy()
    return sample

In [21]:
def generate_hypothesis(iteration: int,
                        obj: pd.Series, 
                        train_data: pd.DataFrame, 
                        other_data: pd.DataFrame, 
                        sample_size: int, 
                        hypothesis_criterion: str,
                        sample_type:str,
                        verbose: bool,
                        other_data_size: int, 
                        alpha: float):
        
        print(f'iteration: {iteration}')
        # Генерация семпла данных, по которым будут вычисляться гипотезы
        # Параметр sample_type определяет область, из которой будут генерироваться гипотезы
        #To-do-list
        #В данном месте нужно заменить места генерации выборки на функцию generate_random_sample
        #Функцию генерации локальной области для объекта, нужно поднять на один уровень выше(в mining_step)
        #иначе локальная область будет высчитываться в каждом процессе
        #В mining_step нужно высчитать один раз область и дальше передать как параметр в дальнейшую функцию
        if sample_type == 'random':
            inds = np.random.RandomState().choice(train_data.index, replace=False, size=sample_size)
            sample = train_data.loc[inds, :].copy()
            sample = sample.append(obj)
        elif sample_type == 'local':
            #генерация семпла данных из локальной области
            area = find_opt_local_area(obj=obj, train_data=train_data, num_iters=100)
            sample = generate_local_sample(d=area, train_data=train_data,
                                           sample_size=int(sample_ratio * train_pos.shape[0])
                                          )
            sample.append(obj)
            #sample = generate_local_sample(obj=obj, train_data=train_data, sample_size=sample_size)
            
        else:
            print('Не задали тип семплирования')
            return None
        
        d = get_similarity_sample_repr(sample)
        if verbose:
            print('got feature represantation for sample')
        
        d_other_objects = is_included_in_repr(d, train_data=other_data)
        #print(f'got {len(d_other_objects)} d_other_objects')
        #print(f'thresh for hypothesis = {int(other_data_size * alpha)}')
        
#         if verbose:
#             print('got objects that is included in sample represantation')
        #!!!!!!!
        # Тут момент такой, что вроде если d_other_objects тоже подходит, так как в этой области только объекты одного класса
        # а объектов другого просто нет
        if d_other_objects is None:
            print('!'*20)
            print('did not find any hypothesis on this iteration')
            return d # эта гипотеза подходит, никого нет из другого класса, можно больше ничего не проверять
        
        ###проверка критерия
        if hypothesis_criterion == 'contr_class':
            if d_other_objects.shape[0] <= int(other_data_size * alpha):
                return d
        elif hypothesis_criterion == 'both_classes':
            #дополнительно смотрим какие объекты target(рассматриваемого на этой итерации) класса попадают в паттерн d
            d_target_objects = is_included_in_repr(d, train_data=train_data)
            if d_other_objects.shape[0] <= int(d_target_objects.shape[0] * alpha):
                return d
        else:
            print('did not get any hyp on this iteration')
            return None

def mining_step(test_obj: pd.Series, 
                train_pos: pd.DataFrame, 
                train_neg: pd.DataFrame,
                num_iters: int, 
                sample_ratio: float, 
                alpha: float, 
                hypothesis_criterion: str, 
                sample_type: str,
                mining_type: str = 'pos', 
                verbose : bool = False, 
                n_jobs : int = 4):
    """
    hypothesis_criterion: 'contr_class', если используем базовый критерий, 
                                когда смотрится пересечение с противоположным классом(старый критерий отбора гипотез)
                           'both_classes', когда интересует пересечение по обоим классам(новый критерий отбора гипотез)
    sample_type: 'random', если берем произвольную выборку интервальных представлений
                 'local', если берем произвольную выборку из локальной области
    
    returns list of hypothesises
    """
    
#     Здесь нужно поставить расчет локальной области объекта, иначе
# для одного объекта в каждом потоке будет пересчитываться область

#Таким образом, здесь мы расчитаем область для объекта, и далее передадим в функцию, где будет происходить случайное семплирование
    
    train_data = train_pos if mining_type == 'pos' else train_neg
    other_data = train_neg if mining_type == 'pos' else train_pos
    other_data_size = train_neg.shape[0] if mining_type == 'pos' else train_pos.shape[0]
    
    sample_size = int(train_data.shape[0] * sample_ratio)
    print('start generating hypothesises')
    
    mining = partial(generate_hypothesis, 
                     obj=test_obj, 
                     train_data=train_data, 
                     other_data=other_data, 
                     sample_size=sample_size, 
                     hypothesis_criterion=hypothesis_criterion,
                     sample_type=sample_type,
                     verbose=verbose, 
                     other_data_size=other_data_size,
                     alpha=alpha
                    )
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        hypothesises = executor.map(mining, range(num_iters))

    hypothesises = [res for res in hypothesises if res is not None]
    return hypothesises

In [30]:
#старая версия, уже не нужна в принципе
def generate_local_area_old(obj: pd.DataFrame):
    eps = 0
    index = obj.index.values[0]
    local_obj = pd.DataFrame(index=[index], columns=obj.columns)
    for key in obj.columns:
        left_val, right_val = obj[key].iloc[0]
        if isinstance(left_val, int):
            eps = 1 if abs(left_val) // 100 == 0 else 100
        elif isinstance(left_val, float):
            eps = 0.01
        left_val, right_val = left_val - eps, right_val + eps 
        local_obj.loc[index, key] = (left_val, right_val)
    return local_obj
#данную функцию переписали, чтобы соответствовала общей логике подсчета
def generate_local_area(obj: pd.Series):
    eps = 0
    index = obj.name
    local_obj = {}
    for feat, val in obj.items():
        left_val, right_val = val
        if isinstance(left_val, int):
            eps = 1 if abs(left_val) // 100 == 0 else 100
        elif isinstance(left_val, float):
            eps = 0.01 if abs(left_val) // 10 == 0 else 100
        left_val, right_val = left_val - eps, right_val + eps
        local_obj[feat] = (left_val, right_val)
    local_obj = pd.Series(local_obj)
    local_obj.name = index
    return local_obj


#здесь будет именно генерация семла в каждом потоке отдельно

def generate_local_sample_old(d: pd.Series, 
                          train_data: pd.DataFrame, 
                          sample_size: int, 
                          frac: float = 0.15,
                          num_iters=10):
    
    iters = num_iters
    sample = None
    d_local_area = obj
    print(f'start generating local sample')
#     Возможно нужно добавить условие на количество объектов в области но не больше какое то количество итераций
    objects_count = 0
    objs_count_thresh = int(train_data.shape[0] * frac)
    while objects_count < objs_count_thresh and iters > 0:
        print(f'itr: {iters}')
        d_local_area = generate_local_area(d_local_area)
        d_local_objects = is_included_in_repr(d=d_local_area, train_data=train_data)
        if d_local_objects is not None:
            objects_count = d_local_objects.shape[0]
            print(f"d_local_object = {len(d_local_objects)}")
            if objects_count > objs_count_thresh:
                inds = np.random.RandomState().choice(d_local_objects.index, replace=False, size=sample_size)
                sample = d_local_objects.loc[inds]
                print(f'found sample')
                break
        iters -= 1
    if sample is None:
        print('Not enough iterations. You shoud use more iterations or choose another obj. Maybe it is outlier.')
        
    return sample

In [55]:
#здесь высчитвается локальная область, которая передается функции генерации семпла
#сейчас расширение области делается в отрыве от самих данных. Нужно предусмотреть ограничения на расширение признака
#Чтобы не получилось, что возраст отрицательный. Можно передавать в качестве параметра вектор минимальных и максимальных значений
#для всех признаков
def find_opt_local_area(obj: pd.Series, train_data: pd.DataFrame,
                       frac: float = 0.15, num_iters=10):
    #нужно запихнуть сюда код, который занимается именно поиском области,
    #а в generate_local_sample оставить именно генерацию семпла по области
    iters = num_iters
    is_found = False
    d_local_area = obj
    objects_count = 0
    objects_count_thresh = int(train_data.shape[0] * frac)
    print(f"start generating local area")
    while objects_count < objects_count_thresh and iters > 0:
        print(f"itr: {iters}")
        d_local_area = generate_local_area(obj=d_local_area)
        d_local_objects = is_included_in_repr(d=d_local_area, train_data=train_data)
        if d_local_objects is not None:
            objects_count = d_local_objects.shape[0]
            print(f"d_local_objects = {objects_count}")
            if objects_count > objects_count_thresh:
                print(f"found opt local area for object")
                is_found = True
                break
        else:
            print(f'found 0 objects in this local area')
        iters -= 1 
    if not is_found:
        print(f"not enough iterations. try more iterations/ Current num_iters = {num_iters}")
    return d_local_area if is_found else None

def generate_local_sample(d: pd.Series, train_data: pd.DataFrame, sample_size: int):
    
    local_objects = is_included_in_repr(d=d, train_data=train_data)
    
    inds = np.random.RandomState().choice(local_objects.index, replace=False, size=sample_size)
    sample = local_objects.loc[inds]
    
    return sample

In [53]:
train_pos.shape[0] * 0.15

150.0

In [54]:
for i, obj in test_sample.iterrows():
    area = find_opt_local_area(obj=obj, train_data=train_pos, num_iters=100)
    if area is not None:
        print('we found area')

start generating local area
itr: 100
found 0 objects in this local area
itr: 99
found 0 objects in this local area
itr: 98
found 0 objects in this local area
itr: 97
found 0 objects in this local area
itr: 96
found 0 objects in this local area
itr: 95
found 0 objects in this local area
itr: 94
found 0 objects in this local area
itr: 93
d_local_objects = 2
itr: 92
d_local_objects = 2
itr: 91
d_local_objects = 5
itr: 90
d_local_objects = 7
itr: 89
d_local_objects = 7
itr: 88
d_local_objects = 7
itr: 87
d_local_objects = 7
itr: 86
d_local_objects = 7
itr: 85
d_local_objects = 9
itr: 84
d_local_objects = 12
itr: 83
d_local_objects = 15
itr: 82
d_local_objects = 18
itr: 81
d_local_objects = 20
itr: 80
d_local_objects = 21
itr: 79
d_local_objects = 24
itr: 78
d_local_objects = 27
itr: 77
d_local_objects = 29
itr: 76
d_local_objects = 35
itr: 75
d_local_objects = 38
itr: 74
d_local_objects = 42
itr: 73
d_local_objects = 45
itr: 72
d_local_objects = 46
itr: 71
d_local_objects = 52
itr: 70
d_lo

In [59]:
sample_ratio * train_pos.shape[0]

10.0

In [60]:
generate_local_sample(d=area, train_data=train_pos, sample_size=int(sample_ratio * train_pos.shape[0]))

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
51285,"(0.26, 0.26)","(72, 72)","(2, 2)","(0.46, 0.46)","(2072, 2072)","(13, 13)","(0, 0)","(2, 2)","(0, 0)","(0, 0)"
74092,"(0.49, 0.49)","(55, 55)","(1, 1)","(0.16, 0.16)","(5200, 5200)","(7, 7)","(2, 2)","(0, 0)","(1, 1)","(1, 1)"
124623,"(0.15, 0.15)","(39, 39)","(0, 0)","(0.33, 0.33)","(4706, 4706)","(7, 7)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
41135,"(0.57, 0.57)","(36, 36)","(0, 0)","(0.32, 0.32)","(1600, 1600)","(8, 8)","(0, 0)","(0, 0)","(1, 1)","(2, 2)"
19480,"(0.41, 0.41)","(26, 26)","(0, 0)","(0.29, 0.29)","(2660, 2660)","(4, 4)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
8912,"(0.2, 0.2)","(31, 31)","(1, 1)","(0.42, 0.42)","(4100, 4100)","(7, 7)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
115405,"(0.0, 0.0)","(42, 42)","(1, 1)","(0.38, 0.38)","(3177, 3177)","(7, 7)","(1, 1)","(2, 2)","(1, 1)","(2, 2)"
60309,"(0.51, 0.51)","(38, 38)","(2, 2)","(0.02, 0.02)","(6666, 6666)","(6, 6)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
17406,"(0.01, 0.01)","(26, 26)","(0, 0)","(0.12, 0.12)","(2200, 2200)","(9, 9)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
143335,"(0.3, 0.3)","(39, 39)","(0, 0)","(0.37, 0.37)","(6539, 6539)","(13, 13)","(0, 0)","(2, 2)","(0, 0)","(6, 6)"


In [48]:
local_neg_objs = is_included_in_repr(d=area, train_data=train_neg)

In [45]:
local_objs = is_included_in_repr(d=area, train_data=train_pos)

In [49]:
len(local_objs), train_pos.shape, len(local_neg_objs)

(153, (1000, 10), 349)

pandas.core.series.Series

In [179]:
# for i, obj in test_sample.iterrows():
#     sample = generate_local_sample(obj, train_data=train_pos, sample_size=3, num_iters=30)

In [183]:
# for i, obj in test_sample.iterrows():
#     print(type(obj))
# #     generate_local_sample(obj=obj, train_data=train_pos, sample_size=3)
#     generated = generate_local_area(obj)

In [184]:
# test_local_obj = transformed_train.sample(1)
# test_local_obj

# generated_obj = generate_local_area(test_local_obj)
# generated_obj

# generate_local_area(generated_obj)

In [185]:
pos_hyps = []
neg_hyps = []

**Генерация гипотез для полож класса занимает 10 мин (с 4 процессами) c 3000 iterations**
**c 1000 iterations - 2 mins**

In [289]:
# %%time
# for i, obj in test_sample.iterrows():
#     print(type(obj))
#     print(f'start mining from pos objects')
#     pos_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg, 
#                            num_iters=num_iters,sample_ratio=sample_ratio, alpha = alpha,
#                            hypothesis_criterion='contr_class',
#                            sample_type='local',
#                            mining_type='pos',
#                            verbose=False, n_jobs=4
#                           )
    
# #     print(f'start mining from neg objects')
# #     neg_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg,
# #                            num_iters=num_iters, sample_ratio=sample_ratio, alpha = alpha, 
# #                            hypothesis_criterion='contr_class',mining_type='neg', 
# #                            sample_type='',
# #                            verbose=True, n_jobs=4
# #                           )

In [62]:
# a = pd.Series({'feat1':(1,1), 'feat2':(0, 1.4), 'feat3':(3, 4)})
# b = pd.Series({'feat1':(1,2), 'feat2':(1, 1.4), 'feat3':(7, 7)})
# c = pd.Series({'feat1':(1,1), 'feat2':(0.8, 1.6), 'feat3':(3, 4)})
# d = pd.Series({'feat1':(2,3), 'feat2':(0, 1.4), 'feat3':(4, 6)})


# print(a.equals(b))
# print(a == b)

# test1 = pd.Series({'feat1':(1,3), 'feat2':(1, 3.4), 'feat3':(1, 1.9)})
# test2 = pd.Series({'feat1':(2,2.2), 'feat2':(2, 2.4), 'feat3':(3, 3.9)})
# test3 = pd.Series({'feat1':(3,3), 'feat2':(4, 4.4), 'feat3':(4, 4.9)})

g1 = pd.Series({'feat1':(1,1), 'feat2':(1.5, 1.5)})
g2 = pd.DataFrame({'feat1':(-1,-1), 'feat2':(0, 0)})
g3 = pd.Series({'feat1':(0.5,0.5), 'feat2':(1, 1)})

test1 = pd.Series({'feat1':(0.1, 1), 'feat2':(-0.5, 0)})
test2 = pd.DataFrame({'feat1':(-0.1, 1), 'feat2':(0.5, 0.8)})
tst = pd.Series({'feat1':(-1, 1), 'feat2':(0, 1.5)})

In [287]:
# for i,obj in sample.iterrows():
#     print(similarity(pd.DataFrame(d).T, obj))

In [286]:
# sample = pd.DataFrame([g1, g2, g3, test1, test2])
# sample

In [248]:
g1

feat1        (1, 1)
feat2    (1.5, 1.5)
dtype: object

In [95]:
type(d), type(sample)

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [186]:
# is_included_in_repr(d, sample)

Тестовая функция провекри np.random.choice

In [40]:
def compute_stat(times, data):
    print(f"times = {times}")
    sample = np.random.RandomState().choice(data, replace=False, size=10)
    print(f"sample = {sample}")
    sample_sum = np.sum(sample)
    print(f"sample sum = {sample_sum}")
    return sample_sum

def run_computing(n_jobs=2):
    data = list(range(100))
    
    compute_func = partial(compute_stat, data=data)
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        results = executor.map(compute_func, list(range(26)))
    
    return results

In [284]:
# t = run_computing(n_jobs=6)